In [2]:
import pandas as pd
import os

In [3]:
assert os.getcwd().endswith('notebooks')
DATASET_DIR = '../dataset/lr+we/dataset'
REPO_DIR = '../dataset/lr+we/repo'
repo_name_of = {
  'AspectJ': 'aspectj',
  'Birt': 'birt',
  'Eclipse_Platform_UI': 'eclipse.platform.ui',
  'JDT': 'eclipse.jdt.ui',
  'SWT': 'eclipse.platform.swt',
  'Tomcat': 'tomcat',
}

In [8]:
import subprocess


def assert_commit_existence(excel_file: str):
  df = pd.read_excel(os.path.join(DATASET_DIR, f'{excel_file}.xlsx'))
  current_sha_of = {}
  for commit in df.commit:
    try:
      checkout_to(commit, repo_name_of[excel_file])
      current_sha_of[str(commit)] = str(commit)
    except subprocess.CalledProcessError as e:
      assert isinstance(e.stderr, str)
      error_lines = e.stderr.split('\n')
      assert error_lines[0] == f'error: short object ID {commit} is ambiguous', error_lines
      # Error is in format --
      # error: short object ID 7a5882a is ambiguous
      # hint: The candidates are:
      # hint:   7a5882abf8 commit 2006-08-14 - Fix for Bug 152170 [FilteredTree] Make it easier to use other viewers
      # hint:   7a5882a826 tree
      # error: pathspec '7a5882a' did not match any file(s) known to git
      #
      # Therefore, first (error, hint1) and last (error, empty) two lines
      # are useless. In the useful lines, words at 1 and 2 indices are useful.
      matches = map(lambda line: line.split()[1: 3], error_lines[2:-2])
      matches = list(filter(lambda match: match[1] == 'commit', matches))
      timestamps = list(map(lambda match: int(subprocess.run(
        f'git show -s --format=%ct {match[0]}',
        shell=True,
        cwd=os.path.join(REPO_DIR, repo_name_of[excel_file]),
        capture_output=True,
        check=True,
        encoding='utf-8',
      ).stdout.strip()), matches))
      assert timestamps[0] == min(timestamps)
      checkout_to(matches[0][0], repo_name_of[excel_file])
      current_sha_of[str(commit)] = matches[0][0]

  return current_sha_of


def checkout_to(commit, repo_name):
  subprocess.run(
    f'git checkout {commit}',
    shell=True,
    cwd=os.path.join(REPO_DIR, repo_name),
    capture_output=True,
    check=True,
    encoding='utf-8',
  )

In [10]:
import json

for filename in repo_name_of.keys():
  if filename in ['AspectJ', 'Birt']:
    continue

  commit_map = assert_commit_existence(filename)
  with open(f'{filename}-commit-map.json', 'w') as commit_map_file:
    json.dump(commit_map, commit_map_file)
  print('Completed', filename)

AssertionError: ["error: pathspec '345f01b' did not match any file(s) known to git", '']